In [ ]:
import os
import argparse
import torch
import torchvision
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import * 
import torch.utils.data
import torchvision.transforms as transforms
import random
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure




In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

**MODEL ARCHITECTURE**

In [ ]:
valid_size = 1024 
batch_size = 128

cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}


class Net(nn.Module):

    model_file="models/default_model.pth"

    def __init__(self, vgg_name):
        super(Net, self).__init__()
        self.features = self._make_layers(cfg[vgg_name])
        self.classifier = nn.Linear(512, 10)

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)

    def save(self, model_file):
        '''Helper function, use it to save the model weights after training.'''
        torch.save(self.state_dict(), model_file)

    def load(self, model_file):
        self.load_state_dict(torch.load(model_file, map_location=torch.device(device)))

        
    def load_for_testing(self, project_dir='./'):
        '''This function will be called automatically before testing your
           project, and will load the model weights from the file
           specify in Net.model_file.
           
           You must not change the prototype of this function. You may
           add extra code in its body if you feel it is necessary, but
           beware that paths of files used in this function should be
           refered relative to the root of your project directory.
        '''        
        self.load(os.path.join(project_dir, Net.model_file))


**VANILLA TRAINING FUNCTION**

In [ ]:
def train_model(net, train_loader, pth_filename, num_epochs):
    
    print("Starting training")

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=1e-2, momentum=0.9, weight_decay=5e-4)

    last_epoch = 0
    mean_loss   = 0
    mean_norm   = 0
    batch_count = 0

    net.train()

    for epoch in range(last_epoch, num_epochs):  

        for param_group in optimizer.param_groups:
          print(f"Learning Rate : {param_group['lr']}")

        print("--- Training epoch {}".format(epoch))
        for i, data in enumerate(train_loader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data[0].to(device), data[1].to(device)
            norm = None
            loss = None
            
            optimizer.zero_grad()
            logits = net(inputs)
            loss   = F.cross_entropy(logits,labels)
            loss.backward()
            optimizer.step()

            mean_loss   += loss.detach().item()
            
            batch_count += 1
            if (batch_count % 10 == 0):
              print(f"* Batch {i+1}/{len(train_loader)} : {mean_loss/batch_count}")
              mean_loss   = 0
              batch_count = 0

        net.save(pth_filename)
        print('Model saved in {}'.format(pth_filename))
        


    net.save(pth_filename)
    print('Model saved in {}'.format(pth_filename))
    

def test_natural(net, test_loader):

    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for i,data in enumerate(test_loader, 0):
            images, labels = data[0].to(device), data[1].to(device)
            # calculate outputs by running images through the network
            outputs = net(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return 100 * correct / total

*Defining main()*

In [ ]:
def get_train_loader(dataset, valid_size=1024, batch_size=32):
    '''Split dataset into [train:valid] and return a DataLoader for the training part.'''

    indices = list(range(len(dataset)))
    train_sampler = torch.utils.data.SubsetRandomSampler(indices[valid_size:])
    train = torch.utils.data.DataLoader(dataset, sampler=train_sampler, batch_size=batch_size)

    return train

def get_validation_loader(dataset, valid_size=1024, batch_size=32):
    '''Split dataset into [train:valid] and return a DataLoader for the validation part.'''

    indices = list(range(len(dataset)))
    valid_sampler = torch.utils.data.SubsetRandomSampler(indices[:valid_size])
    valid = torch.utils.data.DataLoader(dataset, sampler=valid_sampler, batch_size=batch_size)

    return valid

def main(model_name="default_model", epochs=10):

    class Args:

      def __init__(self, mf,ft,ne):
        self.model_file=mf
        self.force_train=ft
        self.num_epochs=ne

    args = Args(model_name+".pth",True,epochs)

    
    net = Net('VGG19')
    net.to(device)

    #### Model training (if necessary)
    if not os.path.exists(args.model_file) or args.force_train:
        print("Training model")
        print(args.model_file)

        train_transform = transforms.Compose([ transforms.RandomHorizontalFlip(),   
                                               transforms.RandomCrop(size=32, padding=4), 
                                               transforms.ToTensor()]) 
        
        cifar = torchvision.datasets.CIFAR10('./data/', download=True, transform=train_transform)
        train_loader = get_train_loader(cifar, valid_size, batch_size=batch_size)
        train_model(net, train_loader, args.model_file, args.num_epochs)
        print("Model save to '{}'.".format(args.model_file))

    #### Model testing
    print("Testing with model from '{}'. ".format(args.model_file))

    # Note: You should not change the transform applied to the
    # validation dataset since, it will be the only transform used
    # during final testing.
    cifar = torchvision.datasets.CIFAR10('./data/', download=True, transform=transforms.ToTensor()) 
    valid_loader = get_validation_loader(cifar, valid_size)

    net.load(args.model_file)

    acc = test_natural(net, valid_loader)
    print("Model natural accuracy (valid): {}".format(acc))

    if args.model_file != Net.model_file:
        print("Warning: '{0}' is not the default model file, "\
              "it will not be the one used for testing your project. "\
              "If this is your best model, "\
              "you should rename/link '{0}' to '{1}'.".format(args.model_file, Net.model_file))

*TRAINING:*

In [ ]:
main("first_model",30)

*Testing the original model*

In [ ]:
net = Net('VGG19')
net.to(device)
net.load("first_model.pth")

cifar = torchvision.datasets.CIFAR10('./data/', download=True, transform=transforms.ToTensor()) 
test_loader = get_validation_loader(cifar, valid_size)

In [ ]:
test_natural(net, test_loader)

**DEFINING WHITE BOX ATTACKS**

In [ ]:
def FGSM(model, x, y, eps=0.01, targeted=False):
    model.eval()            
    x.requires_grad_(True)  

    logits = model(torch.clamp(x,0,1))
    loss   = F.cross_entropy(logits, y)
    loss.backward()

    if not targeted:    x_adv  = x.detach() + eps * torch.sign(x.grad)  
    else:               x_adv  = x.detach() - eps * torch.sign(x.grad)  

    x = x.detach()
    model.zero_grad()
    return x_adv.detach()

In [ ]:
#projecting points onto the lp-norm if outside
def project(x, eps, p=2):
    if p=="inf":
        x = torch.clamp(x,-eps,eps)
    else:
        norms = torch.norm(x.view(x.size(0),-1),dim=1,p=p)
        norms[norms==0] = 1                          
        mask  = (norms > eps)                        
        x[mask] /= norms[mask].view(-1,1,1,1)        
        x[mask] *= eps                               
    return x


def PGD(model, x, y, eps=0.1, stepsize=0.04, iterations=8, p=2, targeted=False):
    model.eval()  

    
    if p=="inf":
        delta = (torch.rand(x.size()) * 2 - 1) * eps   
    else:
        delta = torch.randn_like(x)                                             
        norms = torch.norm(delta.view(delta.size(0),-1), dim=1, p=p)
        norms[norms==0] = 1                                                     
        delta /= norms.view(-1,1,1,1)                                           
        delta *= torch.rand((delta.size(0),1,1,1)).to(device) * eps                       

    delta = delta.to(device)
    
    for i in range(iterations):
        x_adv = x + delta
        x_adv.requires_grad_(True)
        logits = model(torch.clamp(x_adv,0,1))
        loss   = F.cross_entropy(logits,y)
        loss.backward()

        
        if p=="inf":
            gradient = torch.sign(x_adv.grad) * stepsize
        else:
            gradient  = x_adv.grad                                                
            norms     = torch.norm(gradient.view(gradient.size(0),-1),dim=1,p=p)
            norms[norms==0] = 1                                                   
            gradient /= norms.view(-1,1,1,1)                                      
            gradient *= stepsize                                                  

        if not targeted :   x_adv = x_adv.detach() + gradient 
        else:               x_adv = x_adv.detach() - gradient

        
        delta = (x_adv - x).detach()
        delta = project(delta, eps, p)

    x = x.detach()
    model.zero_grad()
    return torch.clamp(x + delta,0,1)


*White box attacks impact on vanilla model*

In [ ]:
#testing 
def test_adv(net, test_loader, attack):
    

    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    for i,data in enumerate(test_loader, 0):
        images, labels = data[0].to(device), data[1].to(device)
        if attack == 'FGSM': 
          images_adv = FGSM(net, images, labels)
        elif attack == 'PGD-l2': 
          images_adv = PGD(net, images, labels)
        else:
          images_adv = PGD(net, images, labels, eps=0.01, stepsize=0.004, p='inf')

        # calculate outputs by running images through the network
        outputs = net(images_adv)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    return 100 * correct / total

In [ ]:
print("Model test accuracy on FGSM-attacked set is", test_adv(net, test_loader,"FGSM"))
print('\n')
print("Model test accuracy on PGD l2-attacked set is", test_adv(net, test_loader,"PGD-l2"))
print('\n')
print("Model test accuracy on PGD linf-attacked set is", test_adv(net, test_loader, "PGD-linf"))

**ADVERSARIAL TRAINING**

We tried adversarial training with multiple combinaison of attacks and ratios introduced on the train loader. The best best set was training on 50% of original images and 50% on images attacked with PGD-linf

In [ ]:

def adv_training(net, train_loader, pth_filename, num_epochs):
    
    print("Starting training")

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=1e-2, momentum=0.9, weight_decay=5e-4)

    last_epoch = 0
    mean_loss   = 0
    mean_norm   = 0
    batch_count = 0

    net.train()

    for epoch in range(last_epoch, num_epochs):  

        for param_group in optimizer.param_groups:
          print(f"Learning Rate : {param_group['lr']}")

        print("--- Training epoch {}".format(epoch))
        for i, data in enumerate(train_loader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data[0].to(device), data[1].to(device)
            adv_inputs=PGD(net,inputs,labels, p='inf')
            concatenated_images = torch.cat((adv_inputs, inputs), 0)
            concatenated_targets = torch.cat((labels,labels), 0)
            norm = None
            loss = None
            
            optimizer.zero_grad()
            logits = net(concatenated_images)
            loss   = F.cross_entropy(logits,concatenated_targets)
            loss.backward()
            optimizer.step()

            mean_loss   += loss.detach().item()
            
            batch_count += 1
            if (batch_count % 10 == 0):
              print(f"* Batch {i+1}/{len(train_loader)} : {mean_loss/batch_count}")
              mean_loss   = 0
              batch_count = 0

        net.save(pth_filename)
        print('Model saved in {}'.format(pth_filename))
        


    net.save(pth_filename)
    print('Model saved in {}'.format(pth_filename))

In [ ]:
def main2(model_name="default_model", epochs=10):

    class Args:

      def __init__(self, mf,ft,ne):
        self.model_file=mf
        self.force_train=ft
        self.num_epochs=ne

    args = Args(model_name+".pth",True,epochs)

    
    net = Net('VGG19')
    net.to(device)

    #### Model training (if necessary)
    if not os.path.exists(args.model_file) or args.force_train:
        print("Training model")
        print(args.model_file)

        train_transform = transforms.Compose([ transforms.RandomHorizontalFlip(),   
                                               transforms.RandomCrop(size=32, padding=4), 
                                               transforms.ToTensor()]) 
        
        cifar = torchvision.datasets.CIFAR10('./data/', download=True, transform=train_transform)
        train_loader = get_train_loader(cifar, valid_size, batch_size=batch_size)
        adv_training(net, train_loader, args.model_file, args.num_epochs)
        print("Model save to '{}'.".format(args.model_file))

    #### Model testing
    print("Testing with model from '{}'. ".format(args.model_file))

    # Note: You should not change the transform applied to the
    # validation dataset since, it will be the only transform used
    # during final testing.
    cifar = torchvision.datasets.CIFAR10('./data/', download=True, transform=transforms.ToTensor()) 
    valid_loader = get_validation_loader(cifar, valid_size)

    net.load(args.model_file)

    acc = test_natural(net, valid_loader)
    print("Model natural accuracy (valid): {}".format(acc))

    if args.model_file != Net.model_file:
        print("Warning: '{0}' is not the default model file, "\
              "it will not be the one used for testing your project. "\
              "If this is your best model, "\
              "you should rename/link '{0}' to '{1}'.".format(args.model_file, Net.model_file))

In [ ]:
main2("robust_model",30)

Testing the adversarially trained model

In [ ]:
net_adv = Net('VGG19')
net_adv.to(device)
net_adv.load("robust_model.pth")

In [ ]:
print("Model test accuracy on original set is:", test_natural(net_adv, test_loader))
print('\n')
print("Model test accuracy on FGSM-attacked set is:", test_adv(net_adv, test_loader,"FGSM"))
print('\n')
print("Model test accuracy on PGD l2-attacked set is:", test_adv(net_adv, test_loader,"PGD-l2"))
print('\n')
print("Model test accuracy on PGD linf-attacked set is:", test_adv(net_adv, test_loader, "PGD-linf"))

Exploring Epsilon effect on model accuracy

In [ ]:
def get_graph(model, epsilons, attack):
  accuracies = []
  for eps in epsilons:
    if attack == "FGSM":      attack_fn = (lambda net,input,label : FGSM(net,input,label,eps,False))
    elif attack == "PGD-l2":  attack_fn = (lambda net,input,label : PGD(net,input,label,eps,eps/10,10,2,False))
    else:                     attack_fn = (lambda net,input,label : PGD(net,input,label,eps,eps/10,10,"inf",False))
    acc = test_adv(model, test_loader, attack_fn)
    accuracies.append(acc)

  return accuracies

*for FGSM attack*

In [ ]:
epsilons = [i*0.01 for i in range(21)]
accuracies_normal = get_graph(net, epsilons, "FGSM")
accuracies_adv    = get_graph(net_adv, epsilons, "FGSM")

In [ ]:
plt.plot(epsilons, accuracies_normal, color="b", label="Vanilla Training")
plt.plot(epsilons, accuracies_adv, color="r", label="Adversarial Training")
plt.ylabel("Accuracy (%)")
plt.xlabel("Epsilon")
plt.legend()
plt.show()

*for PGD-l2 attack*

In [ ]:
epsilons = [i*0.1 for i in range(21)]
accuracies_normal = get_graph(net, epsilons, "PGD-l2")
accuracies_adv    = get_graph(net_adv, epsilons, "PGD-l2")

In [ ]:
plt.plot(epsilons, accuracies_normal, color="b", label="Vanilla Training")
plt.plot(epsilons, accuracies_adv, color="r", label="Adversarial Training")
plt.ylabel("Accuracy (%)")
plt.xlabel("Epsilon")
plt.legend()
plt.show()

*for PGD-linf attack*

In [ ]:
epsilons = [i*0.01 for i in range(21)]
accuracies_normal = get_graph(net, epsilons, "PGD-linf")
accuracies_adv    = get_graph(net_adv, epsilons, "PGD-linf")

In [ ]:
plt.plot(epsilons, accuracies_normal, color="b", label="Vanilla Training")
plt.plot(epsilons, accuracies_adv, color="r", label="Adversarial Training")
plt.ylabel("Accuracy (%)")
plt.xlabel("Epsilon")
plt.legend()
plt.show()

**Black Box Attack 1: NES**

In [ ]:
# fonction NES pour approx du gradient
def NES(model, image, label, sigma, n):
    # Returns estimate of the gradient using NES approach
    img_rows, img_cols, channels = 32, 32, 3
    N = img_rows * img_cols * channels
    x = torch.Tensor(image.reshape(N)).to(device)
    gradient = torch.Tensor(np.zeros((1, channels,img_rows, img_cols ),dtype=np.float32)).to(device)
    cce = nn.CrossEntropyLoss()
    if label.shape!=(1,10):
      label=torch.nn.functional.one_hot(label, num_classes=10).reshape((1,10)).to(device).float()
    for i in range(n):
        
        delta_i = np.random.normal(0,1,N)
        delta_i=delta_i.astype(np.float32)
        delta_i=torch.Tensor(delta_i).to(device)
        theta_pos = x + sigma * delta_i
        theta_neg = x - sigma * delta_i
        theta_pos = torch.Tensor(theta_pos.reshape((1, channels,img_rows, img_cols))).to(device)
        theta_neg = torch.Tensor(theta_neg.reshape((1,channels, img_rows, img_cols))).to(device)
        prediction_pos = model(theta_pos).float()
        prediction_neg = model(theta_neg)

        loss_pos = - cce(label, prediction_pos)
        loss_neg = - cce(label, prediction_neg)

        gradient += loss_pos.item() * delta_i.reshape((1, channels,img_rows, img_cols ))
        gradient -= loss_neg.item() * delta_i.reshape((1, channels,img_rows, img_cols))
    
    gradient /= 2*n*sigma
  
    return gradient

In [ ]:
def projection(sample, norm, epsilon):

    # Clipping perturbation sample to norm ball
    if norm not in [np.inf, 1, 2]:
        raise ValueError('ord must be np.inf, 1, or 2.')

    if norm == np.inf:
        sample = np.clip(sample.cpu(), -epsilon, epsilon)

    return sample

In [ ]:
def PGDBlackBox(model,niter, image, label, eta, epsilon, sigma, n): 
    cost = np.zeros(niter)
    x0 = image
    x_adv = image
    loss=nn.CrossEntropyLoss()
    img_rows, img_cols, channels = 32, 32, 3
    
    for t in range(niter): 

        cost[t] = loss( model(x_adv),label)
        
        x_adv=x_adv.type(torch.FloatTensor)
        GradEstim = NES(model, x_adv, label, sigma, n)
        
        signed_grad = GradEstim.sign().to(device)
        
        x_adv=x_adv.to(device)
        x_adv = x_adv - eta*signed_grad
        
        perturb = x_adv - x0
        
        perturb = torch.Tensor(projection(perturb, np.inf, epsilon)).to(device)
        x_adv = x0 + perturb
        

        x_adv = np.clip(x_adv.cpu(), 0, 255) # ensure it's in the good range
        x_adv=x_adv.to(device)
    print("PGD Black Box done...")
    return (x_adv, cost)

In [ ]:
#parameters
niter = 30
eta = 0.01
epsilon = 0.5
sigma = 0.01
n_ = 50

#number of examples
n_examples=10

#calculate the accuracy
total=0
adv_correct=0
normal_correct=0



with torch.no_grad():
  for i,data in enumerate(test_loader, 0):
    inputs, targets = data[0].to(device), data[1].to(device)
    for i in range(inputs.shape[0]):
      image=inputs[i].reshape((1,3,32,32))
      label=targets[i].reshape((1))
      x_PGD, cost_PGD = PGDBlackBox(net,niter, image, label, eta, epsilon, sigma, n_)
      _,adv_prediction=net(x_PGD).max(1)
      _,prediction=net(image).max(1)
      total+=1
      if adv_prediction.eq(label):
        adv_correct+=1
      if prediction.eq(label):
        normal_correct+=1
      print('\nthe number of generated images :'+str(total))

      if total>=n_examples:
        break
    break

print('Natural accuracy :'+str(normal_correct*100/total))
print('Adverarial accuracy :'+str(adv_correct*100/total))

#if it does not give good results please check your original model  

**Black Box Attack 2: INJECTING RANDOM NOISE**

In [ ]:
#testing 
def test_noised(net):
    

    correct = 0
    total = 0
    valid_loader = get_noised_test_loader(noise=0.1)
    # since we're not training, we don't need to calculate the gradients for our outputs
    for i,data in enumerate(valid_loader, 0):
        images, labels = data[0].to(device), data[1].to(device)

        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    return 100 * correct / total

*Testing injecting noise effect on the models trained previously:*


In [ ]:
print("Vanilla model test accuracy on noised set is:", test_noised(net))
print('\n')
print("Adversarial model accuracy on noised set is:", test_noised(net_adv))


**New Defense Approach: Randomized Network**

This is achieved by injecting gaussian noise to the train loader.
We used this method simultaneously with adversarial training: Regularized Adversarial Training (**RAT**)

In [ ]:
valid_size = 1024
batch_size = 32

class AddGaussianNoise(object):
    def __init__(self, mean, std):
        self.std = std
        self.mean = mean

    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean

    def __repr__(self):
        return self.__class__.__name__ + '(mean={}, std={})'.format(self.mean, self.std)



def get_noised_train_loader(noise, valid_size=valid_size, batch_size=batch_size):
    cifar = torchvision.datasets.CIFAR10('./data/', download=True, transform=transforms.Compose(
        [transforms.RandomHorizontalFlip(), transforms.RandomCrop(size=32, padding=4), transforms.ToTensor(), AddGaussianNoise(0., noise)]))
    train_loader = get_train_loader(cifar, valid_size, batch_size=batch_size)
    return train_loader


def get_noised_test_loader(noise, valid_size=valid_size):
    cifar = torchvision.datasets.CIFAR10('./data/', download=True, transform=transforms.Compose(
        [transforms.ToTensor(), AddGaussianNoise(0., noise)]))
    valid_loader = get_validation_loader(cifar, valid_size, 1)  
    return valid_loader



def training(net, train_loader, num_epochs):
    '''Basic training function'''
    print("Starting training")

    criterion = nn.CrossEntropyLoss()

    optimizer = optim.SGD(net.parameters(), lr=1e-3,
                      momentum=0.9, weight_decay=5e-4)

    train_loss = []
    for epoch in range(num_epochs): 

        loss_per_epoch = 0.0
        for i, data in enumerate(train_loader, 0):
            
          
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()

            
            
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss_per_epoch += loss.item()
            loss.backward()
            optimizer.step()

        epoch_loss = loss_per_epoch / len(train_loader)
        train_loss.append(epoch_loss)
        '''print(f'epoch : {epoch} --- loss : {epoch_loss} ')'''

    return train_loss

def train_network(noise, epochs=20):
    
    net = Net('VGG19')
    net.to(device)
    
    train_loader = get_noised_train_loader(noise=noise)
    
    loss = training(net, train_loader, epochs)
    # VALIDATION
    acc, acc_noised = natural_acc(net, noise)  
    print("Model trained with gaussian noise of {}, final loss of {}.\n "
          "Natural accuracy on classic test data is : {}, natural accuracy on noised test data is {}".format(noise,
                                                                                                             loss, acc,
                                                                                                             acc_noised))
    return [acc, acc_noised]
    
def natural_acc(net, noise):
    valid_noised_loader = get_noised_test_loader(noise=noise)
    valid_loader = get_noised_test_loader(noise=0)

    acc = test_natural(net, valid_loader)
    acc_noised = test_natural(net, valid_noised_loader)
    return acc, acc_noised

def train_multiple(noises, epochs=10):
    a=[]
    b=[]
    
    for n in noises:
      N=train_network(n, epochs)

      a=a.append(train_network(n, epochs)[0])
      b=b.append(train_network(n, epochs)[1])
      
    
    return [a,b]
      

In [ ]:
noiz=[0, 0.01, 0.02, 0.03, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.2, 0.3, 0.5]

In [ ]:
X=train_multiple(noiz, epochs=10)
X

In [ ]:
#num. of epochs
ep=[i for i in range(1,11)]
#training losses
noise_0=[1.2888056789248072, 0.7869450969586257, 0.5858477791935537, 0.4466561506837509, 0.3529048065084947, 0.2696410481721299, 0.21334297112709338, 0.16467761626901442, 0.13839442336233485, 0.10238422836547512]
noise_001=[1.3108496756276455, 0.8074855775828458, 0.6020222639702723, 0.46687756462727087, 0.3660965919407032, 0.2803657855108565, 0.2205072932528085, 0.17605155517647053, 0.14132746913641264, 0.11310747206307811]
noise_002=[1.3144987444329463, 0.8211092451364678, 0.6202640661143696, 0.48477185698582564, 0.38047535907766694, 0.30434330958680805, 0.23690595432439518, 0.19122005370460757, 0.1533006953226389, 0.12615584611289257]
noise_003=[1.3394614815089065, 0.8345390617224533, 0.6299919661044452, 0.49970256258349727, 0.40080077866862446, 0.32065984316700663, 0.2600182276255628, 0.21124502746990542, 0.17196085921304385, 0.1360492542855062]
noise_005=[1.3632234056193715, 0.8770833308987956, 0.6791906547577532, 0.5516207414337967, 0.45026004612251175, 0.3704747513817425, 0.30685885210663255, 0.2488678347801146, 0.214132045612839, 0.17370501224676685]
noise_006=[1.3568309193962467, 0.8820886028544876, 0.6923485718214925, 0.5666812164362235, 0.4641273476275878, 0.3866256887761111, 0.32287008736761935, 0.26942535708302534, 0.22881253791947445, 0.191705191451971]
noise_007=[1.3793536864566616, 0.9039732478242696, 0.7040254888796946, 0.5778023909794435, 0.48551854992967897, 0.40402435751268084, 0.3382490313910605, 0.2875962384132721, 0.24162909669033905, 0.2047734524334454]
noise_008=[1.3974817660336087, 0.9324690723909731, 0.7387780636490598, 0.6066868591744631, 0.5179838278981146, 0.44347720327017587, 0.3731494717139344, 0.31912902047204317, 0.26516474887684893, 0.2277376352645734]
noise_009=[1.3987721760704344, 0.9462217088036876, 0.758692577919253, 0.6307238738845174, 0.5354199004593743, 0.46307728849813723, 0.3962152064342969, 0.33665184197055603, 0.2893354245561895, 0.2530678095822822]
noise_01=[1.4119416697596818, 0.962810063307617, 0.7738572548293818, 0.6498156562885531, 0.5514199158418888, 0.48181529172314763, 0.4093757550372632, 0.3485679674156346, 0.3055674814777893, 0.2662686526709663]
noise_02=[1.5014243485256398, 1.124355860623871, 0.9570515760793007, 0.8484151301665839, 0.7688898360277608, 0.6969107354119429, 0.6326694576900664, 0.5781421898763839, 0.5320383878333517, 0.4826074579400926]
noise_03=[1.5731349808184014, 1.2407874354422288, 1.1008662357975034, 1.0111463333484318, 0.9263936278673655, 0.8700584016501864, 0.8125378827422682, 0.7716610119243138, 0.7230966089988051, 0.6761001867359562]
noise_05=[1.6954003872239145, 1.4403352342268596, 1.3289815649575147, 1.259240535059293, 1.1964331646124418, 1.1480693524371246, 1.10686845560622, 1.075230516459255, 1.0390504901507258, 1.0049954271643555]

In [ ]:
figure(figsize=(10, 6), dpi=80)
plt.plot(ep, noise_0, label="training without noise")
plt.plot(ep, noise_001, label="injecting noise=0.01")
plt.plot(ep, noise_002, label="injecting noise=0.02")
plt.plot(ep, noise_003, label="injecting noise=0.03")
plt.plot(ep, noise_005, label="injecting noise=0.05")
plt.plot(ep, noise_006, label="injecting noise=0.06")
plt.plot(ep, noise_007,  label="injecting noise=0.07")
plt.plot(ep, noise_008, label="injecting noise=0.08")
plt.plot(ep, noise_009, label="injecting noise=0.09")
plt.plot(ep, noise_01, label="injecting noise=0.1")
plt.plot(ep, noise_02, label="injecting noise=0.2")
plt.plot(ep, noise_03, label="injecting noise=0.3")
plt.plot(ep, noise_05, label="injecting noise=0.5")

#plt.plot(epsilons, accuracies_grad, color="g", label="Gradient Minimization")
plt.ylabel("Training Loss")
plt.xlabel("Epochs")
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.1),
          ncol=2, fancybox=True, shadow=True)
plt.show()

In [ ]:
test_accuracy=X[0]
noised_test_accuracy=X[1]

In [ ]:
figure(figsize=(8, 4), dpi=80)
plt.plot(noiz, noised_test_accuracy, label="Accuracy on noised test set")
plt.plot(noiz, test_accuracy, label="Accuracy on original test set")
plt.ylabel("Accuracy")
plt.xlabel("Injected noise")
plt.legend(loc='best',
          ncol=1, fancybox=True, shadow=True)
plt.show()

Injecting noise with sigma=0.06 achieves the best accuracy on both test and noised-test sets.
Now we train a model adversarially by introducing a noise in the training loader and we test the module robustness.

In [ ]:
def main3(model_name="default_model", epochs=10):

    class Args:

      def __init__(self, mf,ft,ne):
        self.model_file=mf
        self.force_train=ft
        self.num_epochs=ne

    args = Args(model_name+".pth",True,epochs)

    
    net = Net('VGG19')
    net.to(device)

    #### Model training (if necessary)
    if not os.path.exists(args.model_file) or args.force_train:
        print("Training model")
        print(args.model_file)

         
        
        train_loader_noised = get_noised_train_loader(noise=0.06
                                                      )
        adv_training(net, train_loader_noised, args.model_file, args.num_epochs)
        print("Model save to '{}'.".format(args.model_file))

    #### Model testing
    print("Testing with model from '{}'. ".format(args.model_file))

    # Note: You should not change the transform applied to the
    # validation dataset since, it will be the only transform used
    # during final testing.
    cifar = torchvision.datasets.CIFAR10('./data/', download=True, transform=transforms.ToTensor()) 
    valid_loader = get_validation_loader(cifar, valid_size)

    net.load(args.model_file)

    acc = test_natural(net, valid_loader)
    print("Model natural accuracy (valid): {}".format(acc))

    if args.model_file != Net.model_file:
        print("Warning: '{0}' is not the default model file, "\
              "it will not be the one used for testing your project. "\
              "If this is your best model, "\
              "you should rename/link '{0}' to '{1}'.".format(args.model_file, Net.model_file))

In [ ]:
main3("RAT_model",30)

*Testing the model trained by injecting noise*

In [ ]:
net_rat = Net('VGG19')
net_rat.to(device)
net_rat.load("RAT_model.pth")

In [ ]:
print("Model test accuracy on original set is:", test_natural(net_rat, test_loader))
print('\n')
print("Model test accuracy on FGSM-attacked set is:", test_adv(net_rat, test_loader,"FGSM"))
print('\n')
print("Model test accuracy on PGD l2-attacked set is:", test_adv(net_rat, test_loader,"PGD-l2"))
print('\n')
print("Model test accuracy on PGD linf-attacked set is:", test_adv(net_rat, test_loader, "PGD-linf"))

**TESTING MODEL ARCHITECTURE IMPACT ON MODEL ROBUSTNESS**

In this part, we trained 4 variants of VGG model (VGG11, VGG13, VGG16 and VGG19). We used adversarial training under the same conditions (same optimizer settings, number of epochs..)

This following dictionary is used to build the models:

In [ ]:
cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}


Training and savind the models:

In [ ]:
def main4(model_name="model", epochs=10):

    '''
    #### Parse command line arguments 
    parser = argparse.ArgumentParser()
    parser.add_argument("--model-file", default=Net.model_file,
                        help="Name of the file used to load or to sore the model weights."\
                        "If the file exists, the weights will be load from it."\
                        "If the file doesn't exists, or if --force-train is set, training will be performed, "\
                        "and the model weights will be stored in this file."\
                        "Warning: "+Net.model_file+" will be used for testing (see load_for_testing()).")
    parser.add_argument('-f', '--force-train', action="store_true",
                        help="Force training even if model file already exists"\
                             "Warning: previous model file will be erased!).")
    parser.add_argument('-e', '--num-epochs', type=int, default=10,
                        help="Set the number of epochs during training")
    args = parser.parse_args()
    '''

    class Args:

      def __init__(self, mf,ft,ne):
        self.model_file=mf
        self.force_train=ft
        self.num_epochs=ne
    for key in cfg:

      args = Args(key+".pth",True,epochs)

    #### Create model and move it to whatever device is available (gpu/cpu)
      net = Net(key)
      net.to(device)

    #### Model training (if necessary)
      if not os.path.exists(args.model_file) or args.force_train:
        print("Training model")
        print(args.model_file)

        train_transform = transforms.Compose([ transforms.RandomHorizontalFlip(),   
                                               transforms.RandomCrop(size=32, padding=4), 
                                               transforms.ToTensor()]) 
        
        cifar = torchvision.datasets.CIFAR10('./data/', download=True, transform=train_transform)
        train_loader = get_train_loader(cifar, valid_size, batch_size=batch_size)
        adv_training(net, train_loader, args.model_file, args.num_epochs)
        print("Model save to '{}'.".format(args.model_file))

    #### Model testing
      print("Testing with model from '{}'. ".format(args.model_file))

    # Note: You should not change the transform applied to the
    # validation dataset since, it will be the only transform used
    # during final testing.
      cifar = torchvision.datasets.CIFAR10('./data/', download=True, transform=transforms.ToTensor()) 
      valid_loader = get_validation_loader(cifar, valid_size)

      net.load(args.model_file)

      acc = test_natural(net, valid_loader)
      print("Model natural accuracy (valid): {}".format(acc))

      if args.model_file != Net.model_file:
        print("Warning: '{0}' is not the default model file, "\
              "it will not be the one used for testing your project. "\
              "If this is your best model, "\
              "you should rename/link '{0}' to '{1}'.".format(args.model_file, Net.model_file))

In [ ]:
main4(epochs=10)

*Testing the models*

In [ ]:
o=[]
f=[]
p1=[]
p2=[]
for key in cfg:
  print('for the'+key+"-model:")

  path=key+".pth"
  net = Net(key)
  net.to(device)
  net.load(path)

  print("Model test accuracy on original set is:", test_natural(net, test_loader))
  o.append(test_natural(net, test_loader))
  print("Model test accuracy on FGSM-attacked set is:", test_adv(net, test_loader,"FGSM"))
  f.append(test_adv(net, test_loader,"FGSM"))
  print("Model test accuracy on PGD l2-attacked set is:", test_adv(net, test_loader,"PGD-l2"))
  p1.append(test_adv(net, test_loader,"PGD-l2"))
  print("Model test accuracy on PGD linf-attacked set is:", test_adv(net, test_loader, "PGD-linf"))
  p2.append(test_adv(net, test_loader, "PGD-linf"))
  print('\n')

*Plotting the results*

In [ ]:
M=['VGG11','VGG13','VGG16','VGG19']

figure(figsize=(12, 6), dpi=80)
plt.plot(M, o, label="Accuracy on original test set")
plt.plot(M, f, label="Accuracy on FGSM-attacked test set")
plt.plot(M,p1, label="Accuracy on PGD l2-attacked test set")
plt.plot(M, p2, label="Accuracy on PGD ling-attacked test set")
plt.ylabel("Accuracy")
plt.xlabel("Models")
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.1),
          ncol=2, fancybox=True, shadow=True)
plt.show()